In [1]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, CSVLogger
from keras.layers import Conv2D, MaxPooling2D, Flatten
import keras.backend.tensorflow_backend as KTF
import matplotlib.pyplot as plt
import tensorflow as tf

Using TensorFlow backend.


In [2]:
batch_size = 50   # バッチサイズ
num_classes = 10  # 分類クラス数　数字認識なので10
epochs = 5 #  学習の繰り返し回数　

# MNISTデータの読み込み
```
(images, labels), (timages, tlabels) = mnist.load_data()
images = images.reshape(60000, 784).astype('float32')
timages = timages.reshape(10000, 784).astype('float32')
images /= 255  # tensorflow と違って keras で読み込んだ MNISTデータは画素が１バイトの整数表現されている
timages /= 255
labels = keras.utils.to_categorica(labels,10)  # keras の MNISTのラベルデータは整数
tlabels = keras.utils.to_categorica(tlabels,10)```

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)
images = mnist.train.images
labels = mnist.train.labels
timages = mnist.test.images
tlabels = mnist.test.labels

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [4]:
images = images.reshape(-1,28,28,1)
timages = timages.reshape(-1,28,28,1)

### Tensorboard 用の設定

In [10]:
old_session = KTF.get_session ()   # tensorboardを使うのに必要な設定

# モデル定義


In [19]:
session = tf. Session ('')     # tensorboardを使うのに必要な設定
KTF.set_session (session)      # tensorboardをわないなら不要
KTF.set_learning_phase (1)     # 

model = Sequential()
model.add(Conv2D(32,kernel_size=(5, 5),padding='same',
                 activation='relu', input_shape=(28,28,1)))
model.add(MaxPooling2D(pool_size=(2, 2)),)
model.add(Conv2D(64,kernel_size=(5, 5),padding='same',
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [20]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 28, 28, 32)        832       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 14, 14, 64)        51264     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 3136)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 1024)              3212288   
_________________________________________________________________
dropout_2 (Dropout)          (None, 1024)              0         
__________

# 学習の定義

In [21]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0),
              metrics=['accuracy'])

Optimizerについては
https://keras.io/ja/optimizers/

In [22]:
es = EarlyStopping(monitor='val_loss', patience=2)   #  検証用データのロスが改善されなくなったら2エポック後に停止
tb_cb = keras.callbacks.TensorBoard(log_dir='tblog', histogram_freq=1, write_graph=True)
csv_logger = CSVLogger('training.log')
hist = model.fit(images, labels,
                 batch_size=batch_size,
                 epochs=epochs,
                 verbose=1,
                 validation_split=0.1, # 1 % は検証用に利用する
                 callbacks=[es, csv_logger,tb_cb])
                 # callbacks=[es, csv_logger])

Train on 49500 samples, validate on 5500 samples
Epoch 1/5
19200/49500 [==========>...................] - ETA: 163s - loss: 0.2342 - acc: 0.9253

KeyboardInterrupt: 

In [ ]:
score = model.evaluate(timages, tlabels, verbose=0)
print('test loss:', score[0])
print('test acc:', score[1])

# plot results
loss = hist.history['loss']
val_loss = hist.history['val_loss']
acc = hist.history['acc']
val_acc = hist.history['val_acc']

epochs = len(loss)
plt.plot(range(epochs), loss, marker='.', label='loss')
plt.plot(range(epochs), val_loss, marker='.', label='val_loss')
plt.plot(range(epochs), acc, marker='.', label='acc')
plt.plot(range(epochs), val_acc, marker='.', label='val_acc')
plt.legend(loc='best')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('acc')
plt.show()

# 隠れ層を１つ追加

In [ ]:
model2 = Sequential()
model2.add(Dense(1024, activation='relu', input_dim=784))
# model2.add(Dropout(0.2))
model2.add(Dense(10, activation='softmax'))

In [ ]:
model2.summary()

In [ ]:
model2.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0),
              metrics=['accuracy'])

In [ ]:
es = EarlyStopping(monitor='val_loss', patience=2)   #  検証用データのロスが改善されなくなったら2エポック後に停止
tb_cb2 = keras.callbacks.TensorBoard(log_dir='tblog2', histogram_freq=10)
csv_logger = CSVLogger('training2.log')
hist = model2.fit(images, labels,
                 batch_size=batch_size,
                 epochs=epochs,
                 verbose=1,
                 validation_split=0.1, # 1 % は検証用に利用する
#                 callbacks=[es, csv_logger,tb_cb2])
                callbacks=[es, csv_logger])

In [ ]:
timages = mnist.test.images
tlabels = mnist.test.labels# evaluate model
score = model2.evaluate(timages, tlabels, verbose=0)
print('test loss:', score[0])
print('test acc:', score[1])

# plot results
loss = hist.history['loss']
val_loss = hist.history['val_loss']
acc = hist.history['acc']
val_acc = hist.history['val_acc']

epochs = len(loss)
plt.plot(range(epochs), loss, marker='.', label='loss')
plt.plot(range(epochs), val_loss, marker='.', label='val_loss')
plt.plot(range(epochs), acc, marker='.', label='acc')
plt.plot(range(epochs), val_acc, marker='.', label='val_acc')
plt.legend(loc='best')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('acc')
plt.show()

In [ ]:
score = model2.evaluate(images, labels, verbose=0)
print('test loss:', score[0])
print('test acc:', score[1])
